# <font color="#003660">Applied Machine Learning for Text Analysis (M.184.5331)</font>


# <font color="#003660">Session 6: Solving Tasks with Prompting LLMs</font>

# <font color="#003660">Notebook 1: Getting Familar with Ollama</font>

<center><br><img width=256 src="https://raw.githubusercontent.com/olivermueller/aml4ta-2021/main/resources/dag.png"/><br></center>

<p>

<div>
    <font color="#085986"><b>By the end of this lesson, you ...</b><br><br>
        ... you know how to use Ollama for prompting LLMs, <br>
        ... how to use its generate and chat api.
    </font>
</div>
</p>

The following content is heavily inspired by the following excellent sources:


* [Raschka (2024): Building a Large Language Model (From Scratch)](https://www.manning.com/books/build-a-large-language-model-from-scratch)
* [HuggingFace (2024): NLP Course](https://huggingface.co/learn/nlp-course/)
* [Huggingface (2024): Open-Source AI Cookbook](https://huggingface.co/learn/cookbook/index)
* [Prompt Engineering Guide](https://www.promptingguide.ai/)
* [Ollama](https://github.com/ollama/ollama)

## Ollama? What is that? Why aren't we using HuggingFace?

The answer is simple: We will use **Large** Langage Models (LLMs) - They are **large**.

Therefore, we will use ollama, which is a simple package running LLMs and making them accessible via API. --> This allows us all to use really large LMs concurrently.

In HuggingFace you can simply use a [HuggingFace Pipeline](https://huggingface.co/docs/transformers/main_classes/pipelines) 

## Setup

First we need to setup a conda environment.
Open the terminal, type the following command and hit enter.
```
conda create -n session_06 python=3.11
```
You will be asked to proceed in the terminal. Answer with "y" and hit ENTER.

```
conda activate session_06
```
You can copy the commands below (don't copy the hashtag).

In [ ]:
# This command creates a new conda environment called session_06 including Python 3.11, JupyterLab, and ipywidgets.
# conda create -n session_06 python=3.11 jupyterlab ipywidgets

If you are using Visual Studio Code select the python environment "session_06" as kernel for the notebook:
<div>
<img src="https://code.visualstudio.com/assets/docs/datascience/jupyter-kernel-management/noterbook-kernel-picker.gif" width="500"/>
</div>

If not, run the command ``jupyter lab`` in your opened terminal. You browser usually will open automatically and provide the jupyter lab environment.

After that, run the following code cell:

In [ ]:
!pip install ollama

## Text Completion with Ollama

### Setting up a Client

Today we will use the python implementation of [Ollama](https://ollama.com/).

Ollama is a package that allows to create, manage, modify and prompt LLMs using a pre-implemented API architecture. --> No API coding, yeah!

If you want to learn more about Ollama refer to the [Ollama Website](https://ollama.com/) and the [Ollama GitHub](https://github.com/ollama/ollama).

In [ ]:
from ollama import Client

We define a Client für connecting to one of the sodalab computers, which is already running ollama.

In [ ]:
host='http://131.234.154.103:11434'
client = Client(host=host)

The client can list all models as well as create, delete, modify LLMs.
The client also allows to generate and chat using LLMs.

Now let's list all models that are available in our ollama instance.

In [ ]:
model_list = client.list()
print(model_list)

As you can see, there is many different information provided for every model.
We only want to know the names for loading a model:

In [ ]:
model_names = [x["model"] for x in model_list["models"]]
for name in model_names:
    print(name)

We start with the ``"gemma2:27b-text-q4_0"`` model, which is a mid-size LLM developed by Google and can run in inference mode on our sodalab computers.

In [ ]:
model_name = "gemma2:27b-text-q4_0"

### Generating text with Gemma

Ollama ``.generate`` uses a specific ``model`` to complete text based on an input prompt.

In [ ]:
prompt = "We will, we will rock you\nWe will, we will rock you\n"
response = client.generate(
    model=model_name, 
    prompt=prompt,
    options={"seed": 42, "num_predict": 50} # standard temperature is 0.7
)
response["context"] = response["context"][:3] + ["..."] + [response["context"][-1]] # ignore that, it is only for visualization

The response object contains the actual text ``"response"``as well as other information, such as ``"prompt_eval_count"`` (number of prompt tokens), ``"eval_count"`` (number of answer tokens),  ``"context"`` (tokenized prompt and answer).

In [ ]:
print("\033[93mJSON response:\033[0m")
print(response)
print()
print("\033[93mOur input prompt:\033[0m")
print(prompt)
print("\033[93mGenerated text:\033[0m")
print(response["response"])

You can also add all options available in the [model API](https://github.com/ollama/ollama/blob/main/docs/modelfile.md#valid-parameters-and-values), such as ``num_predict`` regulating the number of tokens to predict, ``temperature`` of the model, or the ``seed`` for reproducibility.

In [ ]:
prompt = "We will, we will rock you\nWe will, we will rock you\n"
response = client.generate(
    model=model_name, 
    prompt=prompt,
    options={"seed": 42, "num_predict": 50, "temperature": 0.0}
)
print("\033[93mGenerated text:\033[0m")
print(response["response"])

In [ ]:
prompt = "We will, we will rock you\nWe will, we will rock you\n"
response = client.generate(
    model=model_name, 
    prompt=prompt,
    options={"seed": 42, "num_predict": 50, "temperature": 3}
)
print("\033[93mGenerated text:\033[0m")
print(response["response"])

### Prompting Basics

#### Prompting VS Prompt Engineering

**Prompting** can be referred to as passing downstream tasks as textual prompts, unambiguous instructions reformulated to solve like the training data, to LLMs without further retraining while **Prompt (template) engineering** denotes the development of the most appropriate prompt to solve a task ([Kaltenpoth and Müller, 2024](https://aisel.aisnet.org/wi2024/91/); [Liu et al., 2023](https://doi.org/10.1145/3560815)).

Let's start with the process of prompt engineering.
First, let's write a function that generates a response using the ``options``, and ``model_name`` we have already defined and only receives a prompt as input.
Second, we write another function that prints the response.

In [ ]:
def generate_response(prompt, model_name=model_name, options={"seed": 42, "num_predict": 50}):
    response = client.generate(
        model=model_name, 
        prompt=prompt,
        options=options
    )
    return response["response"]

def print_response(prompt, model_name=model_name, options={"seed": 42, "num_predict": 50}):
    print("\033[93mPrompt:\033[0m")
    print(prompt)
    response = generate_response(prompt, model_name, options)    
    print("\033[93mGenerated text:\033[0m")
    print(response)

In [ ]:
prompt = "What did Thomas Edison invent?"
print_response(prompt)

As we can see the answer is not helpful at all. Using the recommendations of [Liu et al. (2023)](https://doi.org/10.1145/3560815) we reformulate it as written in a text describing Alber Einstein:

In [ ]:
prompt = "Thomas Edison invented "
print_response(prompt)

Wow, it's working. Let's try another example.

In [ ]:
prompt = "In which year ended the second World War?"
print_response(prompt)

Hmm... the model completes a possible exam question, but we wanted an answer.

In [ ]:
prompt = "Q: In which year ended the second World War?\nA: " # provide Q: A: format
print_response(prompt)

Another example are maths questions.

In [ ]:
prompt = "What is 25/4? "
print_response(prompt)

Let's try Q&A.

In [ ]:
prompt = "Q: What is 25/4?\nA: "
print_response(prompt)

#### From Zero-Shot Prompting to Few-Shot Prompting.

What we have done so far is called *zero-shot prompting* ([Liu et al., 2023](https://doi.org/10.1145/3560815);[Radford et al., 2019](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)). Prompting for an answer directly. This mostly works in easy cases but not with more complex tasks.

Let's look at an example, where we want to get the sentiment of a very short movie "review".

In [ ]:
prompt = "'I love this movie' is a "
print_response(prompt)

Not helpful. Now lets try to give the model an example, which es referred to as *one-shot prompting* ([Brown et al., 2020](https://papers.nips.cc/paper_files/paper/2020/file/1457c0d6bfcb4967418bfb8ac142f64a-Paper.pdf)).

In [ ]:
prompt = "'I hate this movie' is a negative comment.\n'I love this movie' is a"
print_response(prompt)

Let's try a more complex math problem.

In [ ]:
prompt = "What is 11x4/2?"
print_response(prompt)

Not helpful. Let's try it with a one-shot example.

In [ ]:
prompt = "What is 7x5? 35 What is 11x4/2? "
print_response(prompt)

That's not working, too. Let's use *few-shot prompting*, which is simply using (one or) more examples ([Liu et al., 2023](https://doi.org/10.1145/3560815);[Brown et al., 2020](https://papers.nips.cc/paper_files/paper/2020/file/1457c0d6bfcb4967418bfb8ac142f64a-Paper.pdf)).

In [ ]:
prompt = "What is 7x8? 56 What is 9x9? 81 What is 7x5? 35 What is 11x4/2? "
print_response(prompt)

Wow, it worked. So let's briefly summarize what we learned:

* *Promping* refers to passing (unambiguously defined) natural language instructions to an LLM.
* *Prompt engineering* refers to defining the most appropriate prompt for a task.
* *Zero-shot prompting is just prompting the model without examples.
* *Few-Shot (One-shot) prompting* referst to prompting the model with one or more examples of the task.

As most of you will know ChatGPT, what is different in this models answer compared to those of ChatGPT?

Write it as commend below and share your answer with the seminar.

In [ ]:
# Write here your answer to the questions: What is different in this models answer compared to those of ChatGPT?
#

### Foundation Models VS Chat (Instruction) Models

![Foundation Models VS Chat Models](imgs/llms.png)

(Image adapted from ([Raschka (2024)](https://www.manning.com/books/build-a-large-language-model-from-scratch)))

As visible in the image above, a foundation model is generated by *pretraining*, as those model you trained in the last session.

After pretraining a model, you can fine-tune it to follow instructions or in a conversational manner ([Ouyang et al., 2022](https://proceedings.neurips.cc/paper_files/paper/2022/file/b1efde53be364a73914f58805a001731-Paper-Conference.pdf)).

This is what was done with ChatGPT, Claude and other models ([OpenAI, 2022](https://openai.com/index/chatgpt/);[Ganguli et al., 2022](https://doi.org/10.48550/arXiv.2209.07858)).

## Chatting with Ollama

### Setup the chat API

Let's just load the imports and client again:

In [ ]:
from ollama import Client

In [ ]:
proteus_host='http://131.234.154.103:11434'
client = Client(host=proteus_host)

Ollama provides the ``/chat/`` endpoint for chatting with instruction or conversational models.

We also need to load another version of the gemma model: ``gemma2:27b``, which refers to the 27 billion parameter instruction following model version of [Gemma2](https://ollama.com/library/gemma2:27b).

In [ ]:
model_name = "gemma2:27b"

In Ollama python, the ``.chat`` needs messages in the format provided below, which is similar to the OpenAI API format.

The messages need to be a list of dictionaries that contain a 'role' and 'content' of type string.

In [ ]:
messages = [
    {
        'role': 'user',
        'content': "How are you?",
    },
]
response = client.chat(model=model_name, messages=messages, options={"seed": 42, "num_predict": 50})
print("Ollama respones:")
print(response)

The ollama response contains diverse information on the processing, besides the responses ``"message"`` and its ``"content"``.

In [ ]:
print("Ollama response message:")
print(response['message']['content'])

Let's write two functions for simplification again:

In [ ]:
def generate_chat_response(messages, model_name=model_name, options={"seed": 42, "num_predict": 50}):
    response = client.chat(
        messages=messages,
        model=model_name, 
        options=options
    )
    return response['message']['content']

def print_chat_response(messages, model_name=model_name, options={"seed": 42, "num_predict": 50}):
    print("\033[93mConversation history:\033[0m")
    for message in messages:
        print(f"{message['role']}: {message['content']}")
    response = generate_chat_response(messages, model_name, options)    
    print("\033[93mAnswer:\033[0m")
    print(response)

The messages list needs to start with a message with the ``"system"`` role (e.g, "You are a helpful AI assistant that answers questions.") or a ``"user"`` role.

The system message or *system prompt* mostly contains general instructions that preceed task specific details ([Zhang et al., 2024](https://doi.org/10.48550/arXiv.2410.14826))

In [ ]:
messages = [
    {
        'role': 'system',
        'content': "You are an AI chatbot giving sarcastic answers.",
    },
    {
        'role': 'user',
        'content': "How are you?",
    },
]
print_chat_response(messages=messages)

If there is a message history, the messages should to be alternating ``"user"`` and ``"assistant"``.

In [ ]:
messages = [
    {
        'role': 'system',
        'content': "You are an AI chatbot giving sarcastic answers.",
    },
    {
        'role': 'user',
        'content': "How are you?",
    },
    {
        'role': 'assistant',
        'content': "I looooove being an AI chatbot! Always being asked the same questions over and over again. It's the best!",
    },
    {
        'role': 'user',
        'content': "What do you love the most?",
    },
]
print_chat_response(messages=messages)

### Chain-of-Thought Prompting and Extensions

Let's think of a more complex math riddle, probably for five graders ([Williams and Huckle, 2024](https://doi.org/10.48550/arXiv.2405.19616)):

In [ ]:
my_message = "Sally (a girl) has 3 brothers. Each brother has 2 sisters. How many sisters does Sally have?"
messages = [
    {
        'role': 'system',
        'content': "You are a helpful and honest AI chatbot that follows user instructions and answers questions honestly and helpfully.",
    },
    {
        'role': 'user',
        'content': my_message,
    },
]
print_chat_response(messages=messages, options={"seed": 42})

As we can see, the model answers wrong, while thinking the right way.

The problem is that language models predict the next token by the probability of the previous context ([Shanahan et al., 2024](https://doi.org/10.1038/s41586-023-06647-8)).

This can lead to higher probabilities for the wrong tokens if they are learned from shortcut answers in the training data.

To prevent this, [Wei et al. (2022)](https://proceedings.neurips.cc/paper_files/paper/2022/file/9d5609613524ecf4f15af0f7b31abca4-Paper-Conference.pdf) introduced *Chain-of-Thought (CoT) prompting*, which asks the model to reason step-by-step before answering the question.

![Chain-of-Thought Prompting](imgs/cot.png)


As visible in this illustraion from the Paper of Wei et al. (2022), you can see that they apply few-shot prompting to incorporate the CoT reasoning into the model ([Wei et al., 2022](https://proceedings.neurips.cc/paper_files/paper/2022/file/9d5609613524ecf4f15af0f7b31abca4-Paper-Conference.pdf)).

Current models have already be trained for CoT reasoning. Therefore, we only need to instruct the model to think step-by-step.

In [ ]:
my_message = "Sally (a girl) has 3 brothers. Each brother has 2 sisters. How many sisters does Sally have? Please think step-by-step."
messages = [
    {
        'role': 'system',
        'content': "You are a helpful and honest AI chatbot that follows user instructions and answers questions honestly and helpfully.",
    },
    {
        'role': 'user',
        'content': my_message,
    },
]
print_chat_response(messages=messages, options={"seed": 42})

Wow, the answer is right now. Amazing!

So lets think about another example.

``"When I was 6 my sister was half my age. Now I’m 70 how old is my sister? Please think step-by-step."``

In [ ]:
if False:
    import torch
    from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, set_seed

    set_seed(42)
    model_path = "google/gemma-2-2b-it"
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    quantization_config = nf4_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16
    ) # bitsandbytes only support specific CPU models
    model = AutoModelForCausalLM.from_pretrained(
        model_path, 
        quantization_config=quantization_config,
        device_map="auto"
    )
    model.eval()

Sometimes CoT Prompting just leads to wrong answers, as in this example below.

Please just look at the outputs and don't remove the ``if False:``, as the code can't run on your computer.
For this example you need to install [PyTorch](https://pytorch.org/get-started/locally/), as well as transformers and bitsandbytes. (We will learn more about this libraries in the next session.)

Then *self-consistency* is a helpful approach of prompting ([Wang et al. 2022](https://doi.org/10.48550/arXiv.2203.11171)).

In [ ]:
if False:    
    set_seed(0)
    my_message = "When I was 6 my sister was half my age. Now I’m 70 how old is my sister? Please think step-by-step."

    messages = [
        {
            'role': 'user',
            'content': my_message,
        },
    ]

    chat_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    model_inputs = tokenizer(chat_prompt, return_tensors='pt').to("cuda") # this won't work on CPU

    outputs = model.generate(
        **model_inputs,
        max_new_tokens=256,
        temperature=0.7,
        do_sample=True,
    )
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print("""user
When I was 6 my sister was half my age. Now I’m 70 how old is my sister? Please think step-by-step.
* Step 1:  Find your sister's age when you were 6. 
* Step 2: Find the difference in your ages.
* Step 3:  Add that difference to your current age. 

Here's how it works:

* **Step 1:** You were 6 years old, and your sister was half your age, so she was 6 / 2 = 3 years old.

* **Step 2:** The difference in your ages is 6 - 3 = 3 years.

* **Step 3:**  Add that difference to your current age: 70 + 3 = 73 years old. 

**Answer:** Your sister is 73 years old. 


""")

Self-consistency uses CoT prompting with a more diverse decoding strategy, that replaces *greedy* decoding with a majority vote as visible in the provided in the paper of ([Wang et al. (2022)](https://doi.org/10.48550/arXiv.2203.11171)).

![Self-consistency](imgs/selfcon.png)

Let's do this with our example.

In [ ]:
if False:    
    set_seed(0)
    my_message = "When I was 6 my sister was half my age. Now I’m 70 how old is my sister? Please think step-by-step."

    messages = [
        {
            'role': 'user',
            'content': my_message,
        },
    ]

    chat_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
    model_inputs = tokenizer(chat_prompt, return_tensors='pt').to("cuda") # this won't work on CPU

    beam_outputs = model.generate(
        **model_inputs,
        max_new_tokens=256,
        temperature=0.7, # same as ollama
        do_sample=True,
        num_return_sequences=3,
    )

In [ ]:
if False:    
    for beam_output in beam_outputs:
        print(tokenizer.decode(beam_output, skip_special_tokens=True))
        print("-" * 25)
        print("-" * 25)
print("""user
When I was 6 my sister was half my age. Now I’m 70 how old is my sister? Please think step-by-step.
* Think about how old you were when you were 6.  
* Think about how much older your sister is than you.  
* Do these things to figure out how old she is now. 


**Here's the breakdown:**

* **Step 1:** When you were 6, your sister was half your age, meaning she was 6 / 2 = 3 years old. 
* **Step 2:**  This means your sister is 3 years younger than you.
* **Step 3:** You are now 70 years old.
* **Step 4:** To find your sister's age, subtract the age difference from your current age, which is 70 - 3 = 67.

**Answer:** Your sister is 67 years old. 

-------------------------
-------------------------
user
When I was 6 my sister was half my age. Now I’m 70 how old is my sister? Please think step-by-step.
*Remember the question asks for your sister's age, not your age*

**Step 1:** When you were 6, your sister was half your age, which means she was 6 / 2 = 3 years old.

**Step 2:**  The age difference between you and your sister is 6 - 3 = 3 years.

**Step 3:** Now you are 70 years old.

**Step 4:** Since the age difference remains the same, your sister is 70 - 3 = 67 years old.


**Answer:** Your sister is 67 years old. 

-------------------------
-------------------------
user
When I was 6 my sister was half my age. Now I’m 70 how old is my sister? Please think step-by-step.
* When you were 6, your sister was half your age. 
* Therefore, your sister was 6/2 = 3 years old.
* Now you are 70 years old. 
* The age difference between you and your sister is 70-6 = 6 years.
* Therefore, your sister is 70-6 = 64 years old.



**Answer:** Your sister is 64 years old. 

-------------------------
-------------------------

""")

As we can see, there is one wrong answer, but the majority vote would be "67 years", which is right. Using this decoding strategy with a tree-like structure results in *Tree-of-Thoughts (ToT)* ([Yao et al., 2023](https://doi.org/10.48550/arXiv.2305.10601)), which is out of scope in most cases due to its heavy computation requirements.

#### You can run this cells again!

While we can work with majority votings, we can also change the reasoning that is used for generating answers.

In [ ]:
model_name = "gemma2:2b"
my_message = "India is larger than Russia."
messages = [
    {
        'role': 'system',
        'content': "You are a helpful and honest AI chatbot that follows user instructions and answers questions honestly and helpfully.",
    },
    {
        'role': 'user',
        'content': my_message,
    },
]
print_chat_response(model_name=model_name, messages=messages, options={"seed": 0})

As we can see, the answer is wrong, while the reasoning (after the zero-shot answer) is right.

So let's adress this by first generating some knowledge using a one-shot prompt:

In [ ]:
model_name = "gemma2:2b"
my_message = """
Input: Glasses always fog up.
Knowledge: Condensation occurs on eyeglass lenses when water vapor from your sweat, breath, and ambient humidity lands on a cold surface, cools, and then changes into tiny drops of liquid, forming a film that you see as fog. Your lenses will be relatively cool compared to your breath, especially when the outside air is cold.
Input: Greece is larger than mexico.
Knowledge: Greece is approximately 131,957 sq km, while Mexico is approximately 1,964,375 sq km, making Mexico 1,389% larger than Greece.
Input: India is larger than Russia.
Knowledge:
"""
messages = [
    {
        'role': 'system',
        'content': "You are a helpful and honest AI chatbot that follows user instructions and answers questions honestly and helpfully. You just provide knowledge.",
    },
    {
        'role': 'user',
        'content': my_message,
    },
]
response = generate_chat_response(messages=messages, model_name=model_name, options={"seed": 0}).strip()
print(response)

Using this as an input, we now use one-shot prompting to gather an answer:

In [ ]:
model_name = "gemma2:2b"
my_message = f"""
Input: Greece is larger than mexico.
Knowledge: Greece is approximately 131,957 sq km, while Mexico is approximately 1,964,375 sq km, making Mexico 1,389% larger than Greece.
Answer: Greece is smaller than Mexico.
Hypothesis: India is larger than Russia.
Knowledge: {response}
Answer: """
messages = [
    {
        'role': 'system',
        'content': "You are a helpful and honest AI chatbot that follows user instructions and answers questions honestly and helpfully.",
    },
    {
        'role': 'user',
        'content': my_message,
    },
]
print_chat_response(model_name=model_name, messages=messages, options={"seed": 0})

Wow, the model answers right, based on the knowledge.

This is called *generated knowledge prompting* invented by [Liu et al. (2022)](https://doi.org/10.48550/arXiv.2110.08387).

Let's summarize:

You just learned:
* Zero-shot prompting often fails in terms of correctness ([Radford et al., 2019](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)).
* One-shot or few-shot prompting can improve this in easy settings  ([Liu et al., 2023](https://doi.org/10.1145/3560815);[Brown et al., 2020](https://papers.nips.cc/paper_files/paper/2020/file/1457c0d6bfcb4967418bfb8ac142f64a-Paper.pdf)).
* Chain-of-Thought (CoT) prompting can improve this in more complex settings ([Wei et al., 2022](https://proceedings.neurips.cc/paper_files/paper/2022/file/9d5609613524ecf4f15af0f7b31abca4-Paper-Conference.pdf)).
* Self-consistency, Tree-of-Thoughts or generated knowledge prompting can further improve model responses ([Liu et al., 2022](https://doi.org/10.48550/arXiv.2110.08387);[Wang et al., 2022](https://doi.org/10.48550/arXiv.2203.11171);[Yao et al., 2023](https://doi.org/10.48550/arXiv.2305.10601)).

But most of the Chain-of-Thought improvements are given to mathematical or true-false problems.

The question is: What happens when we want to solve real(-world) tasks? (As you may remember, the topic of this session is "Solving Tasks with prompting LLMs".)

How can we use these prompting techniques, how do we use system prompts? How to reason within a conversation?